# Artificial Neural Network Demonstration
This notebook demonstrates the usage of ANN implemented from scratch in the `tensorfio` package.

## Import Libraries

In [29]:
import pandas as pd

# Custom ANN library
from tensorfio.Layer import Dense
from tensorfio.Sequential import Sequential

## Import Dataset
Using the breast cancer dataset for demonstration.

In [30]:
# Import breast cancer dataset
from sklearn.datasets import load_breast_cancer

breast_cancer_data = load_breast_cancer()
df = pd.DataFrame(breast_cancer_data.data, columns=breast_cancer_data.feature_names)
df['target'] = pd.Series(breast_cancer_data.target)
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [31]:
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

## Split Training and Test Set

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[breast_cancer_data.feature_names], df['target'], test_size=0.2, random_state=42, stratify=df['target'])

In [33]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(455, 30) (114, 30) (455,) (114,)


## Scale the Dataset

In [34]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Implement Artificial Neural Network

### First Model
Output layer: 2-neuron, softmax activation  
Loss function: Binary crossentropy

In [35]:
# Note: input_shape adds an input layer
model = Sequential([
    Dense(3, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='relu'),
], name='First model')

model.add(Dense(2, activation='softmax'))

model.summary()

 Model Summary
--------------------------------------
 Layer (type) | Output Shape | Param #
 Dense        | (3,)         | 93      
 Dense        | (3,)         | 12      
 Dense        | (2,)         | 8       
Total params: 113


In [36]:
# Compile the model
model.compile(optimizer='sgd', loss='crossentropy', metric='accuracy')

In [37]:
# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, lr=0.01, verbose=True, random_state=42, patience=5)

Epoch 1/200
455/455 [==============================] - loss: 1.0834 - accuracy: 0.7104
Epoch 2/200
455/455 [==============================] - loss: 0.3773 - accuracy: 0.8875
Epoch 3/200
455/455 [==============================] - loss: 0.3096 - accuracy: 0.9042
Epoch 4/200
455/455 [==============================] - loss: 0.2747 - accuracy: 0.9083
Epoch 5/200
455/455 [==============================] - loss: 0.2497 - accuracy: 0.9271
Epoch 6/200
455/455 [==============================] - loss: 0.2234 - accuracy: 0.9333
Epoch 7/200
455/455 [==============================] - loss: 0.1945 - accuracy: 0.9396
Epoch 8/200
455/455 [==============================] - loss: 0.1701 - accuracy: 0.9458
Epoch 9/200
455/455 [==============================] - loss: 0.1546 - accuracy: 0.9563
Epoch 10/200
455/455 [==============================] - loss: 0.1427 - accuracy: 0.9563
Epoch 11/200
455/455 [==============================] - loss: 0.1334 - accuracy: 0.9646
Epoch 12/200
455/455 [===================

455/455 [==============================] - loss: 0.0628 - accuracy: 0.9812
Epoch 29/200
455/455 [==============================] - loss: 0.0619 - accuracy: 0.9833
Epoch 30/200
455/455 [==============================] - loss: 0.0609 - accuracy: 0.9854
Epoch 31/200
455/455 [==============================] - loss: 0.0600 - accuracy: 0.9854
Epoch 32/200
455/455 [==============================] - loss: 0.0592 - accuracy: 0.9854
Epoch 33/200
455/455 [==============================] - loss: 0.0583 - accuracy: 0.9854
Epoch 34/200
455/455 [==============================] - loss: 0.0575 - accuracy: 0.9854
Epoch 35/200
455/455 [==============================] - loss: 0.0566 - accuracy: 0.9854
Epoch 36/200
455/455 [==============================] - loss: 0.0558 - accuracy: 0.9854
Epoch 37/200
455/455 [==============================] - loss: 0.0551 - accuracy: 0.9854
Epoch 38/200
455/455 [==============================] - loss: 0.0543 - accuracy: 0.9854
Epoch 39/200
455/455 [=======================

In [38]:
# Evaluate the model
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        43
           1       0.99      0.97      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



### Second Model
Output layer: Single neuron, sigmoid activation  
Loss function: Mean squared error

In [39]:
model2 = Sequential([
    Dense(2, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='relu'),
    Dense(3, activation='relu'),
    Dense(1, activation='sigmoid')
], name='Second model')

model2.summary()

 Model Summary
--------------------------------------
 Layer (type) | Output Shape | Param #
 Dense        | (2,)         | 62      
 Dense        | (3,)         | 9       
 Dense        | (3,)         | 12      
 Dense        | (1,)         | 4       
Total params: 87


In [40]:
# Compile the model
model2.compile(optimizer='sgd', loss='mse', metric='accuracy')

In [41]:
# Train the model
model2.fit(X_train, y_train, epochs=200, batch_size=32, lr=0.01, verbose=True, random_state=42, patience=5)

Epoch 1/200
455/455 [==============================] - loss: 0.2265 - accuracy: 0.6301
Epoch 2/200
455/455 [==============================] - loss: 0.1534 - accuracy: 0.7729
Epoch 3/200
455/455 [==============================] - loss: 0.1209 - accuracy: 0.8292
Epoch 4/200
455/455 [==============================] - loss: 0.1086 - accuracy: 0.8479
Epoch 5/200
455/455 [==============================] - loss: 0.0951 - accuracy: 0.8812
Epoch 6/200
455/455 [==============================] - loss: 0.0813 - accuracy: 0.9104
Epoch 7/200
455/455 [==============================] - loss: 0.0720 - accuracy: 0.9271
Epoch 8/200
455/455 [==============================] - loss: 0.0601 - accuracy: 0.9437
Epoch 9/200
455/455 [==============================] - loss: 0.0468 - accuracy: 0.9646
Epoch 10/200
455/455 [==============================] - loss: 0.0425 - accuracy: 0.9688
Epoch 11/200
455/455 [==============================] - loss: 0.0398 - accuracy: 0.9688
Epoch 12/200
455/455 [===================

In [42]:
# Evaluate the model
y_pred2 = model2.predict(X_test)

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        43
           1       0.99      0.97      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

